In [1]:
lines = open('input1.txt', 'r').readlines()
left_list = []
right_list = []
for line in lines:
	first, second = line.split('   ')
	left_list.append(int(first))
	right_list.append(int(second))
left_list.sort()
right_list.sort()

difference = sum(abs(a-b) for a,b in zip(left_list, right_list))
print(f'The sum of the differences between the numbers in the list is {difference}')

numbers = set(right_list)
counts = {}
for number in numbers:
	counts[number] = right_list.count(number)

similarity_score = sum(number*counts[number] for number in left_list if number in counts)
print(f'The similarity score is {similarity_score}')

The sum of the differences between the numbers in the list is 3508942
The similarity score is 26593248


In [7]:
readings = open('input2.txt', 'r').readlines()

def is_safe(line):
	increasing = True
	decreasing = True
	correct_difference = True
	previous = -1
	for x in line.split(' '):
		x = int(x)
		if previous != -1:
			if x > previous:
				decreasing = False
			if x < previous:
				increasing = False
			if not 1 <= abs(x - previous) <= 3:
				correct_difference = False
		previous = x
	if not correct_difference or not (increasing or decreasing):
		return False
	else:
		return True

safe_readings = sum(1 if safe else 0 for safe in [is_safe(line) for line in readings])
print(f'The number of safe readings is {safe_readings}')

# Brute force the problem dampener
safe_dampened_readings = 0
for reading in readings:
	if is_safe(reading):
		safe_dampened_readings += 1
		continue
	numbers = reading.split(' ')
	for i in range(len(numbers)):
		dampened_reading = ' '.join(numbers[:i] + numbers[i+1:])
		if is_safe(dampened_reading):
			safe_dampened_readings += 1
			break

print(f'The number of safe dampened readings is {safe_dampened_readings}')

The number of safe readings is 213
The number of safe dampened readings is 285


In [15]:
import re
instructions = open('input3.txt', 'r').read()
non_corrupt = re.findall('mul\\((\\d+),(\\d+)\\)',instructions)
print(f'The sum of the real instructions is {sum(int(a)*int(b) for a,b in non_corrupt)}')

# Do or don't
enabled = True
total = 0

finished = False
remaining_instructions = instructions
while not finished:
	first_do = re.search('do\\(\\)', remaining_instructions)
	first_dont = re.search('don\'t\\(\\)', remaining_instructions)
	first_mul = re.search('mul\\((\\d+),(\\d+)\\)', remaining_instructions)

	if first_do is None and first_dont is None and first_mul is None:
		finished = True
		break

	do_hit = dont_hit = mul_hit = len(remaining_instructions)
	if first_do is not None:
		do_hit = first_do.span()[0]
	if first_dont is not None:
		dont_hit = first_dont.span()[0]
	if first_mul is not None:
		mul_hit = first_mul.span()[0]

	if do_hit < dont_hit and do_hit < mul_hit:
		enabled = True
		remaining_instructions = remaining_instructions[first_do.span()[1]:]
		
	if dont_hit < do_hit and dont_hit < mul_hit:
		enabled = False
		remaining_instructions = remaining_instructions[first_dont.span()[1]:]

	if mul_hit < do_hit and mul_hit < dont_hit:
		if enabled:
			total += int(first_mul.group(1)) * int(first_mul.group(2))
		remaining_instructions = remaining_instructions[first_mul.span()[1]:]
print(f'The sum of all conditional muls is {total}')

The sum of the real instructions is 166357705
The sum of all conditional muls is 88811886


In [9]:
word_search = open('input4.txt', 'r').readlines()

directions = [
	(1, 0),
	(1, 1),
	(0, 1),
	(-1, 1),
	(-1, 0),
	(-1, -1),
	(0, -1),
	(1, -1)
]

def look_for_word(row, column):
	if word_search[row][column] != 'X':
		return 0

	words_found = 0
	for direction in directions:
		for i in range(3):
			new_row = row + direction[1]*(i+1)
			new_column = column + direction[0]*(i+1)

			if new_row < 0 or new_row >= len(word_search):
				break
			if new_column < 0 or new_column >= len(word_search[new_row]):
				break

			new_letter = word_search[new_row][new_column]
			if i == 0 and new_letter != 'M':
				break
			if i == 1 and new_letter != 'A':
				break
			if i == 2 and new_letter == 'S':
				words_found += 1

	return words_found

def look_for_xmas(row, column):
	if word_search[row][column] != 'A':
		return 0

	if (not 0 < row < len(word_search) - 1) or (not 0 < column < len(word_search[column]) - 1):
		return 0

	first_diagonal = word_search[row-1][column-1] + word_search[row+1][column+1]
	if (not 'M' in first_diagonal) or (not 'S' in first_diagonal):
		return 0

	second_diagonal = word_search[row-1][column+1] + word_search[row+1][column-1]
	if (not 'M' in second_diagonal) or (not 'S' in second_diagonal):
		return 0

	return 1

words_found = 0
x_mases_found = 0
for row in range(len(word_search)):
	for column in range(len(word_search[row])):
		words_found += look_for_word(row, column)
		x_mases_found += look_for_xmas(row, column)
print(f'Found XMAS {words_found} times')
print(f'Found X-MAS {x_mases_found} times')

Found XMAS 2718 times
Found X-MAS 2046 times


In [1]:
full_input = open('input5.txt', 'r').read()

rules_text, updates_text = full_input.strip().split('\n\n')
rules = []
page_to_first_rule_map = {}
page_to_second_rule_map = {}
for rule in rules_text.split('\n'):
	first_page, second_page = [int(x) for x in rule.split('|')]
	rule_number = len(rules)
	rules.append((first_page, second_page))

	if first_page not in page_to_first_rule_map:
		page_to_first_rule_map[first_page] = set()
	page_to_first_rule_map[first_page].add(rule_number)

	if second_page not in page_to_second_rule_map:
		page_to_second_rule_map[second_page] = set()
	page_to_second_rule_map[second_page].add(rule_number)

def check_if_valid(numbers):
	valid = True
	all_numbers = set(numbers)
	iterated_numbers = set()
	for number in numbers:
		iterated_numbers.add(number)
		if number in page_to_second_rule_map:
			relevant_rules = page_to_second_rule_map[number]
			for rule in relevant_rules:
				first_page, _ = rules[rule]
				if first_page in all_numbers and first_page not in iterated_numbers:
					valid = False
	return valid

updates = [[int(x) for x in line.split(',')] for line in updates_text.split('\n')]
valid_updates = []
for index, numbers in enumerate(updates):
	if check_if_valid(numbers):
		valid_updates.append(index)
		
middle_numbers = 0
for valid_update in valid_updates:
	update_length = len(updates[valid_update])
	middle_numbers += updates[valid_update][update_length // 2]
print(f'The sum of middle pages in the valid updates is {middle_numbers}')

corrected_updates = []
for index, update in enumerate(updates):
	if index in valid_updates:
		continue
	reconstructed = []
	for number in update:
		must_be_before = set()
		if number in page_to_first_rule_map:
			rule_numbers = page_to_first_rule_map[number]
			for rule_number in rule_numbers:
				must_be_before.add(rules[rule_number][1])
		lowest_index = -1
		for other_number in must_be_before:
			if other_number in reconstructed:
				if lowest_index == -1 or reconstructed.index(other_number) < lowest_index:
					lowest_index = reconstructed.index(other_number)
		if lowest_index == -1:
			reconstructed.append(number)
		else:
			reconstructed.insert(lowest_index, number)
	corrected_updates.append(reconstructed)

middle_numbers = 0
for corrected_update in corrected_updates:
	middle_numbers += corrected_update[len(corrected_update) // 2]
print(f'The sum of middle pages in the corrected updates is {middle_numbers}')
	

The sum of middle pages in the valid updates is 4185
The sum of middle pages in the corrected updates is 4480


In [20]:
map_data = open('input6.txt', 'r').read()
directions = [(0, -1), (1, 0), (0, 1), (-1, 0)]
guard = None
obstacles = set()

def move_guard(guard, obstacles):
	position, direction = guard
	moved = False
	while True:
		next_position = (position[0] + directions[direction][0], position[1] + directions[direction][1])
		if next_position not in obstacles:
			return (next_position, direction)
		direction = (direction + 1) % len(directions)

map_lines = map_data.split('\n')
height = len(map_lines)
width = len(map_lines[0])

def is_guard_inside(guard):
	(x, y), _ = guard
	if 0 <= x < width and 0 <= y < height:
		return True
	else:
		return False

for y, line in enumerate(map_lines):
	for x, map_item in enumerate(line):
		if map_item == '#':
			obstacles.add((x, y))
		elif map_item == '^':
			guard = ((x, y), 0)

def find_steps_or_loop(guard, obstacles):
	visited_locations = set()
	location_directions = {}
	while is_guard_inside(guard):
		guard_position, guard_direction = guard
		visited_locations.add(guard_position)
		if guard_position not in location_directions:
			location_directions[guard_position] = set()
		if guard_direction in location_directions[guard_position]:
			return -1
		location_directions[guard_position].add(guard_direction)
		guard = move_guard(guard, obstacles)
	return len(visited_locations)

number_of_steps = find_steps_or_loop(guard, obstacles)
print(f'The guard visits {number_of_steps} locations before leaving the map')

# Find loops
loop_placements = set()
for x,y in visited_locations:
	if (x,y) in obstacles or (x,y) == guard[0]:
		continue
	obstacles_copy = set(obstacles)
	obstacles_copy.add((x,y))
	if find_steps_or_loop(guard, obstacles_copy) == -1:
		loop_placements.add((x,y))

print(f'There are {len(loop_placements)} locations that will set the guard into a loop')

The guard visits 5305 locations before leaving the map
There are 2143 locations that will set the guard into a loop
